# Résolution de Systèmes DAE : Du Simple au Complexe

---

**Objectifs pédagogiques :**
- Comprendre la différence entre ODE et DAE
- Identifier les DAE "simples" (index 1, facilement réductibles) vs "vrais" DAE
- Modéliser deux cas réalistes avec contraintes algébriques
- Implémenter et résoudre des systèmes DAE avec Python

**Prérequis :**
- Connaissances de base en EDO
- Python (NumPy, SciPy, Matplotlib)
- Notions de dynamique des fluides (pour le cas 2)

**Durée estimée :** 60-90 minutes

---

## 📚 Introduction : Qu'est-ce qu'un "vrai" DAE ?

### ODE vs DAE

**ODE (Ordinary Differential Equations)** :
$$
\frac{dy}{dt} = f(t, y)
$$

**DAE (Differential-Algebraic Equations)** :
$$
\begin{align}
\frac{dy}{dt} &= f(t, y, z) \quad \text{(équations différentielles)} \\
0 &= g(t, y, z) \quad \text{(contraintes algébriques)}
\end{align}
$$

### Différents Niveaux de Complexité

| Type | Description | Exemple | Résolution |
|------|-------------|---------|------------|
| **DAE "Simple"** | Contrainte explicite en $z$ | $z = h(y)$ | Substitution directe → ODE |
| **DAE Index 1** | Une dérivation suffit | $0 = g(y, z)$, $\frac{\partial g}{\partial z}$ inversible | Solveur DAE standard |
| **DAE Index 2+** | Plusieurs dérivations nécessaires | Systèmes mécaniques, circuits électriques | Reformulation ou solveur spécialisé |

### Ce que vous allez apprendre

1. **Cas 1 - Réacteur Batch (DAE Simple)** : La contrainte peut être éliminée facilement
2. **Cas 2 - Réservoir avec Vanne (Vrai DAE)** : La contrainte couple plusieurs variables dynamiquement

---

# PARTIE I - CAS SIMPLE : Réacteur Batch Isotherme

## 🏭 Contexte

Réaction chimique simple en batch : $A + B \rightarrow C$

**⚠️ Note importante :** Ce cas illustre un DAE d'index 1 où la contrainte algébrique peut être **facilement éliminée par substitution**. C'est un excellent point de départ pédagogique, mais ce n'est pas un "vrai" DAE complexe.

### Modèle Mathématique

**Équations différentielles :**
$$
\begin{align}
\frac{dN_A}{dt} &= -k \cdot \frac{N_A \cdot N_B}{V} \\
\frac{dN_B}{dt} &= -k \cdot \frac{N_A \cdot N_B}{V} \\
\frac{dN_C}{dt} &= +k \cdot \frac{N_A \cdot N_B}{V}
\end{align}
$$

**Contrainte algébrique :**
$$
V = \frac{N_A + N_B + N_C}{\rho_{mix}}
$$

**💡 Simplification possible :** On peut directement substituer $V$ dans les équations → système ODE pur !

---

In [ ]:
# Imports
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✅ Imports réussis !")

In [ ]:
# Cas 1 : Paramètres du réacteur batch
k = 0.5
rho_mix = 10.0
N_A0, N_B0, N_C0 = 10.0, 15.0, 0.0
V0 = (N_A0 + N_B0 + N_C0) / rho_mix

t_span = (0, 10)
t_eval = np.linspace(t_span[0], t_span[1], 200)

def batch_reactor_dae(t, y, k, rho_mix):
    """DAE simple : contrainte algébrique facilement substituable"""
    N_A, N_B, N_C = y
    V = (N_A + N_B + N_C) / rho_mix  # Contrainte résolue directement
    
    if V <= 0 or N_A < 0 or N_B < 0:
        return np.array([0.0, 0.0, 0.0])
    
    r = k * (N_A * N_B) / (V**2)
    return np.array([-r * V, -r * V, +r * V])

# Résolution
y0 = np.array([N_A0, N_B0, N_C0])
sol_batch = solve_ivp(
    batch_reactor_dae, t_span, y0, args=(k, rho_mix),
    method='Radau', t_eval=t_eval, rtol=1e-6, atol=1e-9
)

print(f"✅ Cas 1 résolu - {sol_batch.nfev} évaluations")

In [ ]:
# Visualisation Cas 1
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(sol_batch.t, sol_batch.y[0], 'b-', linewidth=2, label='$N_A$')
ax1.plot(sol_batch.t, sol_batch.y[1], 'r-', linewidth=2, label='$N_B$')
ax1.plot(sol_batch.t, sol_batch.y[2], 'g-', linewidth=2, label='$N_C$')
ax1.set_xlabel('Temps [s]', fontweight='bold')
ax1.set_ylabel('Quantité [mol]', fontweight='bold')
ax1.set_title('CAS 1 - Réacteur Batch (DAE Simple)', fontweight='bold', fontsize=13)
ax1.legend()
ax1.grid(True, alpha=0.3)

V_batch = (sol_batch.y[0] + sol_batch.y[1] + sol_batch.y[2]) / rho_mix
ax2 = axes[1]
ax2.plot(sol_batch.t, V_batch, 'm-', linewidth=2.5)
ax2.axhline(y=V0, color='k', linestyle='--', label=f'$V_0$={V0:.2f} L')
ax2.set_xlabel('Temps [s]', fontweight='bold')
ax2.set_ylabel('Volume [L]', fontweight='bold')
ax2.set_title('Volume (Contrainte Algébrique)', fontweight='bold', fontsize=13)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 OBSERVATION : Ce cas est un DAE 'simple' car V peut être directement calculé.")
print(f"   On pourrait réécrire le système comme un ODE pur en substituant V.")
print(f"\n→ Passons maintenant à un VRAI DAE plus complexe...\n")

---

# PARTIE II - CAS COMPLEXE : Réservoir avec Contrôle Automatique de Vanne

## 🚰 Contexte Industriel

Un réservoir cylindrique reçoit un débit d'entrée **fluctuant** $Q_{in}(t)$ (variations du processus amont). 

**Objectif** : Maintenir le volume entre deux limites sécuritaires ($V_{min}$ et $V_{max}$) en **ajustant automatiquement l'ouverture de la vanne**.

### Reformulation Correcte en DAE

**❌ ERREUR FRÉQUENTE** (ce que j'ai fait avant) :
```python
# Calculer u explicitement dans la fonction
u = Kp * (h - h_sp) + u_bias  
dV_dt = Q_in - C_v * u * sqrt(2*g*h)
```
→ Ceci est une **reformulation ODE**, pas un DAE !

**✅ VRAIE FORMULATION DAE** :

**États** : $\mathbf{y} = [V, u]$
- $V$ : volume (différentiel)
- $u$ : ouverture de vanne (algébrique - **résolu par le solveur**)

**Système DAE** :
$$
\begin{cases}
\frac{dV}{dt} = Q_{in}(t) - Q_{out}(u, V) & \text{(équation différentielle)} \\
0 = Q_{target}(V) - Q_{out}(u, V) & \text{(contrainte algébrique)}
\end{cases}
$$

où :
- $Q_{out}(u, V) = C_v \cdot u \cdot \sqrt{2gh}$ avec $h = V/A$ (Torricelli)
- $Q_{target}(V)$ : débit cible pour maintenir le volume dans les limites

### Fonction de Débit Cible

La contrainte algébrique impose que le débit de sortie s'ajuste pour maintenir le niveau :

$$
Q_{target}(V) = \begin{cases}
0.1 \cdot Q_{in,moyen} & \text{si } V < V_{min} \text{ (ralentir vidange)} \\
2.0 \cdot Q_{in,moyen} & \text{si } V > V_{max} \text{ (accélérer vidange)} \\
Q_{in,moyen} & \text{sinon (équilibre)}
\end{cases}
$$

**Le solveur DAE cherche la valeur de $u$ qui satisfait** $Q_{target}(V) = Q_{out}(u, V)$ à chaque instant.

---

In [ ]:
# Cas 2 : Paramètres du réservoir avec contrôle automatique de vanne

# Géométrie
A = 2.0  # Surface du réservoir [m²]
g = 9.81  # Gravité [m/s²]

# Vanne
C_v = 0.8  # Coefficient de vanne [m^2.5/s]

# Limites sécuritaires
h_min = 2.5  # Hauteur minimale [m]
h_max = 3.5  # Hauteur maximale [m]
V_min = A * h_min  # Volume minimal [m³]
V_max = A * h_max  # Volume maximal [m³]

# Conditions initiales
V_init = A * 3.0  # Volume initial (h=3m) [m³]
u_init = 0.5  # Ouverture initiale estimée [-]

# Débit d'entrée fluctuant
Q_in_moyen = 0.5  # Débit moyen [m³/s]

def Q_in_fluctuating(t):
    """Débit d'entrée avec fluctuations sinusoïdales"""
    amplitude = 0.25  # Amplitude des fluctuations [m³/s]
    periode = 20.0  # Période [s]
    
    return Q_in_moyen + amplitude * np.sin(2 * np.pi * t / periode)

def Q_target(V):
    """
    Débit de sortie cible en fonction du volume
    
    Cette fonction définit la contrainte algébrique :
    - Si V < V_min : ralentir la vidange
    - Si V > V_max : accélérer la vidange
    - Sinon : équilibrer avec le débit moyen d'entrée
    """
    if V < V_min:
        return 0.1 * Q_in_moyen  # Presque fermer la vanne
    elif V > V_max:
        return 2.0 * Q_in_moyen  # Ouvrir davantage
    else:
        return Q_in_moyen  # Équilibre

print("📊 Paramètres du système réservoir-vanne (modèle DAE corrigé) :")
print(f"  • Surface réservoir : A = {A} m²")
print(f"  • Coefficient vanne : C_v = {C_v} m^2.5/s")
print(f"  • Limites sécuritaires : h ∈ [{h_min}, {h_max}] m")
print(f"  • Volume initial : V(0) = {V_init} m³ (h={V_init/A} m)")
print(f"  • Débit moyen d'entrée : {Q_in_moyen} m³/s")
print(f"\n⚠️ Dans ce modèle, 'u' est une VARIABLE ALGÉBRIQUE")
print(f"   résolue par le solveur DAE, pas calculée explicitement !")

In [ ]:
# ❌ Pourquoi scipy.solve_ivp NE PEUT PAS résoudre ce DAE

print("🚫 TENTATIVE AVEC SCIPY.SOLVE_IVP (VOUÉE À L'ÉCHEC)")
print("="*60)
print("\nPour résoudre ce système avec solve_ivp, il faudrait :")
print("  1. À chaque pas de temps, résoudre la contrainte algébrique")
print("     0 = Q_target(V) - C_v * u * sqrt(2*g*V/A)")
print("     pour trouver u (avec fsolve par exemple)")
print("  2. Puis calculer dV/dt = Q_in - Q_out")
print("\nProblèmes :")
print("  ❌ Très lent (résolution algébrique à chaque évaluation)")
print("  ❌ Instable (fsolve peut échouer ou diverger)")
print("  ❌ Pas de garantie de cohérence globale")
print("  ❌ Le solveur ODE ne 'voit' pas les contraintes")
print("\n💡 SOLUTION : Utiliser un VRAI solveur DAE comme Assimulo IDA")
print("="*60)

In [ ]:
# ✅ RÉSOLUTION AVEC ASSIMULO IDA (Vrai solveur DAE)

print("🔧 Vérification de la disponibilité d'Assimulo...")

try:
    from assimulo.solvers import IDA
    from assimulo.problem import Implicit_Problem
    
    ASSIMULO_AVAILABLE = True
    print("✅ Assimulo est installé !")
    print("\n📦 Installation (si nécessaire) :")
    print("   conda install -c conda-forge assimulo")
    print("   ou pip install assimulo  # (nécessite compilateurs)")
    
except ImportError:
    ASSIMULO_AVAILABLE = False
    print("❌ Assimulo n'est pas installé")
    print("\n📦 Pour installer :")
    print("   conda install -c conda-forge assimulo  # Recommandé")
    print("   ou pip install assimulo  # (nécessite gcc/gfortran + Sundials)")
    print("\n💡 Je vais quand même montrer le code complet ci-dessous")

In [ ]:
# Définition du système DAE pour Assimulo IDA

def reservoir_dae_residual(t, y, yd):
    """
    Résidu du système DAE : F(t, y, dy/dt) = 0
    
    États :
        y = [V, u]
        - V : volume [m³] (différentiel)
        - u : ouverture de vanne [0-1] (algébrique)
    
    Dérivées :
        yd = [dV/dt, 0]
        - dV/dt : taux de variation du volume
        - 0 : pas de dérivée pour u (variable algébrique)
    
    Résidus :
        res[0] = dV/dt - (Q_in - Q_out)  # Équation différentielle
        res[1] = Q_target(V) - Q_out    # Contrainte algébrique
    """
    V, u = y
    dV_dt, _ = yd
    
    # Sécurités
    V = max(V, 0.1)  # Éviter volume négatif
    u = np.clip(u, 0.0, 1.0)  # Saturation de la vanne
    
    # Calculs intermédiaires
    h = V / A
    Q_in = Q_in_fluctuating(t)
    Q_out = C_v * u * np.sqrt(2 * g * h)
    Q_tgt = Q_target(V)
    
    # Résidus (doivent être = 0)
    res_diff = dV_dt - (Q_in - Q_out)  # Équation différentielle
    res_alg = Q_tgt - Q_out             # Contrainte algébrique
    
    return np.array([res_diff, res_alg])

print("✅ Fonction de résidu DAE définie")
print("\nStructure du système :")
print("  • y = [V, u]  - Volume (diff) et ouverture vanne (alg)")
print("  • res[0] = 0  - Bilan de matière : dV/dt = Q_in - Q_out")
print("  • res[1] = 0  - Contrainte : Q_target(V) = Q_out(u, V)")
print("\n💡 Le solveur IDA trouve simultanément V(t) et u(t)")
print("   en satisfaisant les deux équations à chaque instant")

# Configuration et résolution avec IDA

if ASSIMULO_AVAILABLE:
    print("⏳ Configuration du solveur IDA...")
    
    # Conditions initiales
    t0 = 0.0
    y0 = np.array([V_init, u_init])  # [V, u]
    
    # Calcul des dérivées initiales cohérentes
    h0 = V_init / A
    Q_in0 = Q_in_fluctuating(t0)
    Q_out0 = C_v * u_init * np.sqrt(2 * g * h0)
    yd0 = np.array([Q_in0 - Q_out0, 0.0])  # [dV/dt, 0]
    
    # Spécification des variables : 1 = différentielle, 0 = algébrique
    algvar = [1, 0]  # V est différentielle, u est algébrique
    
    # Création du problème DAE
    model = Implicit_Problem(reservoir_dae_residual, y0, yd0, t0)
    model.algvar = algvar
    model.name = 'Réservoir avec contrôle automatique de vanne'
    
    # Configuration du solveur IDA
    sim = IDA(model)
    sim.rtol = 1e-6
    sim.atol = 1e-8
    sim.suppress_alg = True  # Supprime les variables algébriques dans l'erreur
    sim.verbosity = 30  # Niveau de détail modéré
    
    print("✅ Solveur IDA configuré")
    print(f"  • Conditions initiales : V(0) = {y0[0]:.2f} m³, u(0) = {y0[1]:.2f}")
    print(f"  • Variables algébriques : u (ouverture de vanne)")
    print(f"  • Tolérances : rtol={sim.rtol}, atol={sim.atol}")
    
    # Résolution
    print("\n⏳ Résolution du système DAE sur [0, 100] s...")
    t_final = 100.0
    ncp = 500  # Nombre de points de communication
    
    t, y, yd = sim.simulate(t_final, ncp)
    
    print("✅ Résolution terminée !")
    print(f"  • {len(t)} points calculés")
    print(f"  • {sim.statistics['nsteps']} pas de temps")
    print(f"  • {sim.statistics['nfcns']} évaluations de fonction")
    
    # Extraction des résultats
    V_ida = y[:, 0]
    u_ida = y[:, 1]
    h_ida = V_ida / A
    Q_out_ida = C_v * u_ida * np.sqrt(2 * g * h_ida)
    Q_in_ida = np.array([Q_in_fluctuating(ti) for ti in t])
    
    print(f"\n📊 Résultats :")
    print(f"  • Volume : min={V_ida.min():.3f} m³, max={V_ida.max():.3f} m³")
    print(f"  • Hauteur : min={h_ida.min():.3f} m, max={h_ida.max():.3f} m")
    print(f"  • Limites : h_min={h_min} m, h_max={h_max} m")
    print(f"  • Ouverture vanne : min={u_ida.min()*100:.1f}%, max={u_ida.max()*100:.1f}%")
    
    # Vérifier si le niveau reste dans les limites
    violations = np.sum((h_ida < h_min) | (h_ida > h_max))
    print(f"  • Violations des limites : {violations}/{len(t)} ({100*violations/len(t):.1f}%)")
    
else:
    print("⚠️ Assimulo n'est pas disponible - impossible de résoudre le DAE")
    print("   Installez Assimulo pour exécuter cette cellule")

# Visualisation des résultats (Assimulo IDA)

if ASSIMULO_AVAILABLE:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. Hauteur du réservoir avec limites sécuritaires
    ax1 = axes[0, 0]
    ax1.plot(t, h_ida, 'b-', linewidth=2.5, label='Hauteur réelle')
    ax1.axhline(y=h_min, color='r', linestyle='--', linewidth=2, label=f'h_min = {h_min} m')
    ax1.axhline(y=h_max, color='r', linestyle='--', linewidth=2, label=f'h_max = {h_max} m')
    ax1.fill_between(t, h_min, h_max, alpha=0.2, color='green', label='Zone sécuritaire')
    ax1.set_xlabel('Temps [s]', fontweight='bold', fontsize=12)
    ax1.set_ylabel('Hauteur [m]', fontweight='bold', fontsize=12)
    ax1.set_title('1. Hauteur du Réservoir (Maintien Entre Limites)', fontweight='bold', fontsize=13)
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # 2. Ouverture de vanne (variable algébrique résolue par IDA)
    ax2 = axes[0, 1]
    ax2.plot(t, u_ida * 100, 'orange', linewidth=2.5)
    ax2.set_xlabel('Temps [s]', fontweight='bold', fontsize=12)
    ax2.set_ylabel('Ouverture Vanne [%]', fontweight='bold', fontsize=12)
    ax2.set_title('2. Ouverture de Vanne (Variable Algébrique)', fontweight='bold', fontsize=13)
    ax2.set_ylim([-5, 105])
    ax2.grid(True, alpha=0.3)
    ax2.text(0.98, 0.97, 'Calculé par le solveur DAE\n(pas explicitement !)', 
             transform=ax2.transAxes, ha='right', va='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8), fontsize=10)
    
    # 3. Débits
    ax3 = axes[1, 0]
    ax3.plot(t, Q_in_ida, 'g-', linewidth=2, label='$Q_{in}$ (fluctuant)')
    ax3.plot(t, Q_out_ida, 'r-', linewidth=2, label='$Q_{out}$ (ajusté par vanne)')
    ax3.set_xlabel('Temps [s]', fontweight='bold', fontsize=12)
    ax3.set_ylabel('Débit [m³/s]', fontweight='bold', fontsize=12)
    ax3.set_title('3. Débits d\'Entrée et Sortie', fontweight='bold', fontsize=13)
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)
    
    # 4. Volume avec limites
    ax4 = axes[1, 1]
    ax4.plot(t, V_ida, 'm-', linewidth=2.5, label='Volume')
    ax4.axhline(y=V_min, color='r', linestyle='--', linewidth=2, label=f'V_min = {V_min:.1f} m³')
    ax4.axhline(y=V_max, color='r', linestyle='--', linewidth=2, label=f'V_max = {V_max:.1f} m³')
    ax4.fill_between(t, V_min, V_max, alpha=0.2, color='green')
    ax4.set_xlabel('Temps [s]', fontweight='bold', fontsize=12)
    ax4.set_ylabel('Volume [m³]', fontweight='bold', fontsize=12)
    ax4.set_title('4. Volume du Réservoir', fontweight='bold', fontsize=13)
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ RÉSULTAT : Le solveur DAE ajuste automatiquement l'ouverture")
    print("   de la vanne pour maintenir le niveau dans les limites !")
    print(f"\n📊 Performance du contrôle :")
    in_bounds = np.sum((h_ida >= h_min) & (h_ida <= h_max))
    print(f"  • Temps dans la zone sécuritaire : {100*in_bounds/len(t):.1f}%")
    print(f"  • Amplitude des fluctuations de h : {h_ida.max() - h_ida.min():.3f} m")
    print(f"  • Amplitude de contrôle de u : {u_ida.min()*100:.1f}% - {u_ida.max()*100:.1f}%")

else:
    print("⚠️ Visualisation impossible sans Assimulo")

In [ ]:
## 📊 Comparaison : Reformulation ODE vs Vrai DAE

### Ce que nous avons appris

| Aspect | Approche ODE (cell-8 ancienne) | Approche DAE (Assimulo IDA) |
|--------|-------------------------------|----------------------------|
| **Calcul de u** | Explicite : `u = Kp*error + bias` | Implicite : résolu par le solveur |
| **Garantie de contrainte** | ❌ Non, u est juste calculé | ✅ Oui, contrainte satisfaite rigoureusement |
| **Stabilité** | ⚠️ Peut diverger | ✅ Stable |
| **Performance** | ❌ Le contrôle ne fonctionne pas bien | ✅ Maintien précis des limites |
| **Type de système** | ODE reformulé | DAE véritable |

### 🔑 Différence Fondamentale

**Reformulation ODE** :
```python
# Je calcule u, puis j'évalue dV/dt
u = f(V)  # Calcul explicite
dV_dt = g(V, u)  # Injection
```

**Vrai DAE** :
```python
# Le solveur trouve (V, u) simultanément en résolvant :
# Résidu 1 : dV/dt - (Q_in - Q_out) = 0
# Résidu 2 : Q_target(V) - Q_out(u, V) = 0
```

**Le solveur DAE garantit que les deux équations sont satisfaites à chaque instant**, pas seulement approximativement.

---

## ⚠️ LIMITATION CRITIQUE : SciPy n'est PAS un Vrai Solveur DAE

### 🚨 Problème Fondamental

**Ce que nous avons fait** dans le Cas 2 :
- Calculer explicitement les variables algébriques ($h$, $u$, $Q_{out}$) à chaque pas
- Injecter le résultat dans l'équation différentielle $dV/dt$
- Utiliser `solve_ivp` avec Radau (solveur **ODE** implicite)

**Ce qu'un VRAI solveur DAE fait** :
- Résoudre **simultanément** les équations différentielles ET les contraintes algébriques
- Garantir la cohérence des variables algébriques à chaque itération
- Gérer correctement l'index du système

### ❌ Pourquoi le Contrôle Peut Ne Pas Fonctionner Correctement

Avec notre approche `solve_ivp` :

1. **Reformulation ODE approximative** : En calculant $u$ explicitement, on transforme un DAE en ODE, mais cette reformulation peut être **imprécise** ou **instable**
2. **Pas de garantie de cohérence** : Les contraintes algébriques ne sont pas satisfaites rigoureusement
3. **Sensibilité aux paramètres** : Le contrôle peut diverger ou osciller car le solveur ne "voit" pas les contraintes
4. **Régulation vers la consigne impossible** : Le système peut dériver loin de la consigne sans mécanisme de correction algébrique

### ✅ Solution : Utiliser un Vrai Solveur DAE

Pour résoudre correctement le Cas 2, il faut utiliser **Assimulo** (wrapper Python pour Sundials IDA/CVode).

#### Installation

```bash
pip install assimulo
# Note: Nécessite des compilateurs (gcc/gfortran) et Sundials
```

#### Exemple de Reformulation pour IDA

```python
from assimulo.solvers import IDA
from assimulo.problem import Implicit_Problem

def reservoir_dae_residual(t, y, yd):
    """
    Résidu du système DAE : F(t, y, dy/dt) = 0
    
    y = [V, u, h, Q_out]  # États + variables algébriques
    yd = [dV/dt, 0, 0, 0]  # Dérivées (0 pour variables algébriques)
    """
    V, u, h, Q_out = y
    dV_dt, _, _, _ = yd
    
    Q_in = Q_in_fluctuating(t)
    error = h - h_sp
    u_desired = K_p * error + u_bias
    
    # Résidus (doivent être = 0)
    res_V = dV_dt - (Q_in - Q_out)  # Équation différentielle
    res_u = u - np.clip(u_desired, 0, 1)  # Contrainte algébrique
    res_h = h - V / A  # Contrainte algébrique
    res_Q = Q_out - C_v * u * np.sqrt(2 * g * max(h, 0))  # Contrainte algébrique
    
    return np.array([res_V, res_u, res_h, res_Q])

# Définition du problème
y0 = [V_init, 0.5, 3.0, 0.5]  # [V, u, h, Q_out]
yd0 = [0.0, 0.0, 0.0, 0.0]  # Dérivées initiales
algvar = [1, 0, 0, 0]  # 1 = différentielle, 0 = algébrique

problem = Implicit_Problem(reservoir_dae_residual, y0, yd0, t0=0)
problem.algvar = algvar

# Résolution avec IDA
solver = IDA(problem)
solver.rtol = 1e-6
solver.atol = 1e-9
t_out, y_out, yd_out = solver.simulate(100)
```

### 📊 Tableau Comparatif des Solveurs

| Outil | Type | Adapté DAE ? | Installation | Difficulté |
|-------|------|--------------|--------------|------------|
| **scipy.solve_ivp (Radau/BDF)** | ODE implicite | ⚠️ Reformulation ODE uniquement | Facile (`pip install scipy`) | Faible |
| **Assimulo (IDA)** | DAE vrai | ✅ Oui, index 0-3 | Moyenne (compilateurs requis) | Moyenne |
| **PyDy** | Dynamique multibody | ✅ Oui, spécialisé mécanique | Facile | Moyenne |
| **CasADi** | Optimisation/DAE | ✅ Oui, très performant | Moyenne | Élevée |
| **DASSL (Fortran)** | DAE classique | ✅ Oui, référence historique | Difficile | Élevée |

### 💡 Conclusion pour ce Notebook

Ce notebook utilise `solve_ivp` à des **fins pédagogiques** pour :
- ✅ Comprendre la structure des DAE (différentielles + algébriques)
- ✅ Identifier le couplage algébrique
- ✅ Modéliser un système de contrôle réaliste

**Pour une application industrielle réelle**, utilisez **Assimulo IDA** ou un solveur DAE spécialisé.

---

## ⚙️ Points de Vigilance pour les DAE (Suite)

### 2. Index du Système

- **Cas 1** : Index 1 (peut être réduit en ODE)
- **Cas 2** : Index 1 mais avec couplage fort (vrai DAE)

### 3. Conditions Initiales Cohérentes

Pour les DAE, les conditions initiales doivent **satisfaire les contraintes algébriques** :

**Cas 2 :**
- Si on impose $V(0)$, alors automatiquement $h(0) = V(0)/A$
- Et $u(0)$ doit être cohérent avec la loi de contrôle
- Et $Q_{out}(0) = C_v u(0) \sqrt{2gh(0)}$

**⚠️ Erreur courante** : Imposer des conditions initiales incohérentes avec les contraintes.

### 4. Stabilité Numérique

Les systèmes DAE avec contrôle peuvent être raides. Pour un vrai solveur DAE :
- Tolérances strictes (`rtol=1e-6`, `atol=1e-9`)
- Solveur DAE implicite (Assimulo IDA, DASSL)
- Jacobienne analytique si disponible (gain de performance)

---

## 🎓 Exercices Avancés

### Exercice 1 - Contrôleur PI Complet
Améliorez le contrôleur en ajoutant un terme **intégral** :
$$
u(t) = K_p e(t) + K_i \int_0^t e(\tau) d\tau + u_{bias}
$$

**Indice :** Ajouter une variable d'état pour l'intégrale de l'erreur.

### Exercice 2 - Perturbations Aléatoires
Modifiez $Q_{in}(t)$ pour inclure du **bruit blanc** en plus des oscillations :
```python
Q_in = Q_base + amplitude * sin(ωt) + σ * randn()
```

### Exercice 3 - Contrainte d'Inégalité
Ajoutez une contrainte de **niveau maximum** : si $h > h_{max}$, forcer $u = 1$ (vanne pleine ouverte).

### Exercice 4 - Cascade de Réservoirs
Modélisez **2 réservoirs en série** :
- Réservoir 1 : débit d'entrée fluctuant
- Réservoir 2 : reçoit la sortie du réservoir 1
- Contrôle de la vanne du réservoir 2

**Système DAE à 2 équations différentielles + contraintes algébriques.**

---

## 📚 Références

### Documentation
- [scipy.integrate.solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)
- [DAE Systems - APMonitor](https://apmonitor.com/pdc/index.php/Main/SolveDifferentialAlgebraicEquations)

### Livres
1. **Petzold & Ascher** - "Computer Methods for ODEs and DAEs" (référence)
2. **Hairer & Wanner** - "Solving Ordinary Differential Equations II"
3. **Seborg, Edgar, Mellichamp** - "Process Dynamics and Control"

### Applications Industrielles
- Contrôle de niveau dans réservoirs
- Régulation de pression dans pipelines
- Systèmes hydrauliques complexes
- Circuits électriques (index 2-3)
- Dynamique des corps rigides

---

## ✅ Synthèse

**Ce que vous avez appris :**

1. ✅ **Différence entre DAE "simple" et "vrai DAE"**
   - Cas 1 : Contrainte facilement éliminable → quasi-ODE
   - Cas 2 : Couplage algébrique fort → vrai DAE

2. ✅ **Modélisation de systèmes avec contrôle**
   - Variables algébriques (ouverture de vanne)
   - Lois de contrôle (proportionnel)
   - Contraintes physiques (Torricelli)

3. ✅ **Résolution numérique de DAE complexes**
   - Solveur Radau pour stabilité
   - Gestion des contraintes à chaque pas
   - Conditions initiales cohérentes

4. ✅ **Analyse de systèmes dynamiques**
   - Impact des paramètres de contrôle
   - Réponse aux perturbations
   - Stabilité et performance

**Compétences acquises :**
- 🔧 Identifier un "vrai" DAE
- 🎛️ Modéliser des systèmes avec contrôle
- 📊 Résoudre et analyser des DAE complexes
- ⚙️ Optimiser les paramètres de régulation

---

**🎉 Vous maîtrisez maintenant la résolution de DAE du simple au complexe !**